In [26]:
from IPython.core.display import display, HTML

In [27]:
import requests 
from bs4 import BeautifulSoup
from dateutil import parser
from textstat.textstat import textstat

In [28]:
# create ficticous headers to fool the website from percieving the scraper
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36(KHTML, like Gecklo) Chrome/56.0.2924.87 Safari/537.36',
    'referrer': 'https://google.com'
}

In [29]:
url = 'https://blog.frame.io/2018/10/01/womans-experience-cutting-blockbusterrs/'
r = requests.get(url, headers=headers)

In [30]:
r.status_code

200

In [31]:
# extract out the text from my request object
html = r.text.strip()

In [32]:
print(html)

<!DOCTYPE html>

<script>
 var fioPageType = ''; 
var fioPageSlug = ''; 
fioPageType = 'article';
 fioPageSlug = 'womans-experience-cutting-blockbusterrs';
 </script>
<html lang="en-US" prefix="og: http://ogp.me/ns#">
	<head>
		<meta charset="UTF-8">
		<meta name="viewport" content="width=device-width, initial-scale=1">

		<link rel="profile" href="http://gmpg.org/xfn/11">

		

		<title>On Gender and Genre: A Woman’s Experience Cutting Action Blockbusters</title>

<!-- This site is optimized with the Yoast SEO plugin v8.1 - https://yoast.com/wordpress/plugins/seo/ -->
<meta name="description" content="“Mile 22” Editor Melissa Lawson Cheung shares her experience cutting action blockbusters, and what she brings to the cutting room as a woman."/>
<link rel="canonical" href="https://blog.frame.io/2018/10/01/womans-experience-cutting-blockbusterrs/" />
<meta property="og:locale" content="en_US" />
<meta property="og:type" content="article" />
<meta property="og:title" content="On Gender and

In [33]:
# user beautiful soup to parse my html with lxml components
soup = BeautifulSoup(html, 'lxml')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [34]:
# the soup object inherit methods from the beautiful soup class to search lxml

In [35]:
# use the soup.find method to hunt for html elements by class name
header = soup.find(class_='entry-header')
title_html = header.find(class_='post-meta-title')
print(title_html)

<h1 class="post-meta-title">
								On Gender and Genre: A Woman’s Experience Cutting Action Blockbusters							</h1>


In [36]:
# when I call  beautifulsoup object for its contents it returns a list of selected elements as content
print(" The Object type is", type(title_html))
title_html.contents

 The Object type is <class 'bs4.element.Tag'>


['\r\n\t\t\t\t\t\t\t\tOn Gender and Genre: A Woman’s Experience Cutting Action Blockbusters\t\t\t\t\t\t\t']

In [37]:
# I can strip aways all trailing spaces and styling content with strip
title_str = title_html.contents[0].strip()
print(title_str)

On Gender and Genre: A Woman’s Experience Cutting Action Blockbusters


In [38]:
# repeat above steps for each element I want to extract!
author_html = header.find(class_="author-name")
author_str = author_html.find('a').contents[0].strip()
print(author_str)

Lisa McNamara


### Modularize my scraping with functions:

In [83]:
def parse_page(url):
    # URL handling and the intial parse
    r = requests.get(url, headers= headers)
    html = r.text.strip()
    soup = BeautifulSoup(html, 'lxml')
    
    
    
    # Find and parse header content
    header = soup.find(class_='entry-header')
    read_time = extract_read_time(header)
    title = extract_title(header)
    
    author = extract_author(header)
    categories = extract_categories(header)
    
    date = extract_date(header)
    
    dt = parser.parse(date)
    month = dt.strftime("%B")
    weekday = dt.strftime("%A")

    # Body Content
    
    content = soup.find(class_='entry-content')
    word_count = len(content.text.split())
    reading_level = textstat.flesch_kincaid_grade(content.text)
    
    links = content.find_all("a")
    link_count = len(links)
    
    images = content.find_all("img")
    image_count = len(images)
    
    page_data = {
        
        "reading_time": read_time,
        'title': title,
        'date': date,
        'month': month,
        'weekday': weekday,
        'author': author,
        'categories': categories,
        'word_count': word_count,
        'reading_level': reading_level,
        'link_count': link_count,
        'image_count': image_count
    }
    
    return page_data

def extract_read_time(header):
    html_str = header.find(class_='read-time')
    time_str = html_str.contents[0].strip().lower().split()[0]
    time_int = int(time_str)
    return time_int

def extract_title(header):
    html_str = header.find(class_='post-meta-title')
    title_str = html_str.contents[0].strip()
    return title_str

def extract_date(header):
    html_str = header.find(class_='single-post-date')
    date_str = html_str.contents[0].strip()
    return date_str

def extract_author(header):
    html_str = header.find(class_='author-name')
    author_str = html_str.find('a').contents[0].strip()
    return author_str

def extract_categories(header):
    html_str = header.find(class_='single-post-cat')
    categories = html_str.findAll('a')
    cat_names = []
    for cat_link in categories:
        cat_name = cat_link.contents[0].strip().lower()
        cat_names.append(cat_name)
    return cat_names



In [84]:
url = 'https://blog.frame.io/2018/10/01/womans-experience-cutting-blockbusterrs/'
wmn_exp = parse_page(url)

In [85]:
print(wmn_exp)

{'reading_time': 13, 'title': 'On Gender and Genre: A Woman’s Experience Cutting Action Blockbusters', 'date': 'October 1, 2018', 'month': 'October', 'weekday': 'Monday', 'author': 'Lisa McNamara', 'categories': ['behind the scenes'], 'word_count': 2639, 'reading_level': 17.1, 'link_count': 6, 'image_count': 15}


In [86]:
# change object to JSON
import json
export = json.dumps(wmn_exp)

In [87]:
print(export)

{"reading_time": 13, "title": "On Gender and Genre: A Woman\u2019s Experience Cutting Action Blockbusters", "date": "October 1, 2018", "month": "October", "weekday": "Monday", "author": "Lisa McNamara", "categories": ["behind the scenes"], "word_count": 2639, "reading_level": 17.1, "link_count": 6, "image_count": 15}


### Scrape a whole section

In [88]:
articles_store = []

In [89]:
def parse_category(url):
    r = requests.get(url, headers=headers)
    html = r.text.strip()
    soup = BeautifulSoup(html, 'lxml')
    
    # find the elements I want to iterate over
    article_cards = soup.findAll(class_='post-content')
    
    # iterate over articles 
    for article in article_cards:
        title = article.find(class_='post-meta-title')
        link = title.contents[0]['href']
        print('Parsing URL', link)
        page = parse_page(link)
        articles_store.append(page)
        
    next_link = find_next_link(soup)
    
    if next_link is not None:
        print("Next page", next_link)
        parse_category(next_link)
    
    return None

def find_next_link(soup_item):
    bottom_nav = soup_item.find(class_='navigation')
    
    if bottom_nav == None:
        return None
    
    links = bottom_nav.findAll('a')
    next_page = links[-1]
    
    if next_page.contents[0] == 'Next':
        next_link = next_page['href']
        return next_link
    
    return

In [91]:
bts = 'https://blog.frame.io/category/behind-the-scenes/'
parse_category(bts)

Parsing URL https://blog.frame.io/2018/11/19/made-in-frame-cinelab/
Parsing URL https://blog.frame.io/2018/10/01/womans-experience-cutting-blockbusterrs/
Parsing URL https://blog.frame.io/2018/09/10/made-in-frame-searching/
Parsing URL https://blog.frame.io/2018/07/30/inside-mission-impossible-fallout/
Parsing URL https://blog.frame.io/2018/07/16/made-in-frame-film-riot/
Parsing URL https://blog.frame.io/2018/07/02/bts-hotel-artemis/
Parsing URL https://blog.frame.io/2018/06/25/made-in-frame-hereditary/
Parsing URL https://blog.frame.io/2018/06/18/making-incredibles-2/
Parsing URL https://blog.frame.io/2018/05/29/made-in-frame-conan-obrien/
Parsing URL https://blog.frame.io/2018/05/07/madeinframe-kiwi/
Parsing URL https://blog.frame.io/2018/03/26/made-in-frame-new-balance-fearlessly-independent/
Parsing URL https://blog.frame.io/2018/02/26/made-in-frame-get-out/
Next page https://blog.frame.io/category/behind-the-scenes/page/2/
Parsing URL https://blog.frame.io/2017/08/21/editor-as-wri

In [92]:
print(len(articles_store))

21


### Extracting all categories available

In [93]:
articles_store = []

In [94]:
categories = ['post-production', 'color-correction', 'business', 'workflow', 'behind-the-scenes', 'production', 'annocunement']

In [95]:
for category in categories:
    url = 'https://blog.frame.io/category/' + category + '/'
    print('Parsing Category', category)
    parse_category(url)

Parsing Category post-production
Parsing URL https://blog.frame.io/2018/11/05/fcpx-browser-editing/
Parsing URL https://blog.frame.io/2018/09/24/hevc-format-wars/
Parsing URL https://blog.frame.io/2018/09/17/fcpx-shortcuts/
Parsing URL https://blog.frame.io/2018/09/04/creating-video-for-blind-and-deaf/
Parsing URL https://blog.frame.io/2018/08/27/deep-dive-fcpx-audio-tools/
Parsing URL https://blog.frame.io/2018/07/23/tips-for-no-coverage/
Parsing URL https://blog.frame.io/2018/06/11/adr-primer/
Parsing URL https://blog.frame.io/2018/05/14/premiere-batch-syncing/
Parsing URL https://blog.frame.io/2018/03/12/studying-6-editing-masterpieces/
Parsing URL https://blog.frame.io/2018/02/19/avid-media-composer-troubleshooting-tips/
Parsing URL https://blog.frame.io/2018/02/12/animated-masks-after-effects/
Parsing URL https://blog.frame.io/2018/02/07/editorial-style/
Next page https://blog.frame.io/category/post-production/page/2/
Parsing URL https://blog.frame.io/2018/01/31/fcpx-metadata/
Par

Parsing URL https://blog.frame.io/2018/09/24/hevc-format-wars/
Parsing URL https://blog.frame.io/2018/04/30/red-workflows/
Parsing URL https://blog.frame.io/2018/04/02/arri-workflow-premiere/
Parsing URL https://blog.frame.io/2018/03/05/oscar-2018-workflows/
Parsing URL https://blog.frame.io/2018/01/17/remote-dailies-workflow/
Parsing URL https://blog.frame.io/2017/10/04/turbo-charge-fcpx-workflow-davinci-resolve/
Parsing URL https://blog.frame.io/2017/07/31/baby-driver-workflow/
Parsing URL https://blog.frame.io/2017/05/15/sync-clips-in-davinci-resolve/
Parsing URL https://blog.frame.io/2017/03/20/premiere-pro-proxies/
Parsing Category behind-the-scenes
Parsing URL https://blog.frame.io/2018/11/19/made-in-frame-cinelab/
Parsing URL https://blog.frame.io/2018/10/01/womans-experience-cutting-blockbusterrs/
Parsing URL https://blog.frame.io/2018/09/10/made-in-frame-searching/
Parsing URL https://blog.frame.io/2018/07/30/inside-mission-impossible-fallout/
Parsing URL https://blog.frame.io

In [64]:
len(articles_store)

159

In [67]:
articles_store[0]

{'author': 'Sofi Marshall',
 'categories': ['editing'],
 'date': 'November 5, 2018',
 'image_count': 26,
 'link_count': 15,
 'month': 'November',
 'reading_level': 12.4,
 'reading_time': 20,
 'title': 'Clean Up Your Timeline with Browser-Based Editing in Final Cut Pro X',
 'word_count': 4093}

In [70]:
# create a file in the data folder that stores JSON
with open('data/articles.json', 'w') as f:
    json.dump(articles_store, f)

### Computing Analytics

In [71]:
from collections import Counter

In [77]:
# fetch data stored in file
with open('data/articles.json', 'r') as f:
    articles_store = json.loads(f.read())
    
print(articles_store)

[{'reading_time': 20, 'title': 'Clean Up Your Timeline with Browser-Based Editing in Final Cut Pro X', 'date': 'November 5, 2018', 'month': 'November', 'author': 'Sofi Marshall', 'categories': ['editing'], 'word_count': 4093, 'reading_level': 12.4, 'link_count': 15, 'image_count': 26}, {'reading_time': 11, 'title': 'HEVC (H.265): What is it and Why Should You Care?', 'date': 'September 24, 2018', 'month': 'September', 'author': 'Seth Goldin', 'categories': ['codecs', 'workflow'], 'word_count': 2349, 'reading_level': 10.5, 'link_count': 46, 'image_count': 4}, {'reading_time': 15, 'title': 'Edit At The Speed of Thought With FCP X Shortcuts', 'date': 'September 17, 2018', 'month': 'September', 'author': 'Reuben Evans', 'categories': ['editing'], 'word_count': 3091, 'reading_level': 11.3, 'link_count': 48, 'image_count': 27}, {'reading_time': 17, 'title': 'Design for the Edges: Video for the Blind and Deaf', 'date': 'September 4, 2018', 'month': 'September', 'author': 'Katie Hinsen', 'cate

In [96]:
times = []
months= []
weekdays = []
authors = []
categories = []

for article in articles_store:
    
    # calculate the average reading TIime
    times.append(articles['reading_time'])
    average_time = sum(times) / float(len(times))
    average_time = round(average_time,2)
    
    # Posts by month 
    months.append(article['month'])
    month_count = Counter(months)
    
    # Posts by weekday 
    weekdays.append(article['weekday'])
    weekday_count = Counter(weekdays)
    
    # Count by Category
    categories += article['categories']
    category_count = Counter(categories)
    
    # Count by Author
    authors.append(article['author'])
    author_count = Counter(authors)
    

In [97]:
print("Average reading time:", average_time)
print("Posts by month", month_count)
print("Posts by weekday", weekday_count)
print("Posts by category", category_count)
print("Posts by author", author_count)

Average reading time: 20.0
Posts by month Counter({'August': 20, 'November': 17, 'September': 17, 'October': 17, 'June': 14, 'January': 14, 'July': 13, 'December': 13, 'May': 12, 'March': 10, 'February': 7, 'April': 5})
Posts by weekday Counter({'Monday': 87, 'Wednesday': 31, 'Thursday': 18, 'Tuesday': 13, 'Friday': 10})
Posts by category Counter({'editing': 40, 'post-production': 23, 'career': 20, 'storytelling': 15, 'behind the scenes': 14, 'workflow': 12, 'business': 9, 'made in frame': 9, 'codecs': 8, 'color correction': 7, 'production': 6, 'sound': 3, 'cribs': 2, 'design': 2, 'technology': 1})
Posts by author Counter({'Ron Dawson': 16, 'Mark Christiansen': 13, 'Stephen Heleker': 12, 'Jason Boone': 11, 'Lisa McNamara': 11, 'Sofi Marshall': 9, 'David Kong': 8, 'Scott Strandberg': 7, 'Yuri Baranovsky': 6, 'Reuben Evans': 5, 'Clara Lehmann': 5, 'T Payton': 5, 'Seth Goldin': 3, 'Jeff Hinton': 3, 'Hilda Saffari': 3, 'Jarle Leirpoll': 3, 'Zack Arnold': 3, 'Ryan Charles': 2, 'Alexander Hu

In [98]:
# store stats in a file
stats = {
    'reading_time':average_time,
    'num_articles':len(articles_store)
}

with open('data/stats.json', 'w') as f:
    json.dump(stats,f)
    
with open('data/month.json', 'w') as f:
    json.dump(month_count, f)
    
with open('data/weekday.json', 'w') as f:
    json.dump(weekday_count, f)
    
with open('data/category.json', 'w') as f:
    json.dump(category_count, f)
    
with open('data/author.json', 'w') as f:
    json.dump(author_count, f)